# Lab: Ridge Regression and the Lasso

**Information:**  
We are using the book 'G. James et al. -  An Introduction to Statistical Learning (with Applications in Python)'. You can find a copy of it for free [here](https://www.statlearning.com/).

This lab is based on Section 6.6 Lab 2: Ridge Regression and the Lasso (p. 273 - 280) of the book.

For this lab, we use the `Hitters` data set. It contains Major League Baseball Data from the 1986 and 1987 seasons. 
For more information about the data set, click [here](https://www.rdocumentation.org/packages/ISLR/versions/1.2/topics/Hitters).

**Goal of the Lab:**    
In this lab, we aim to fit a ridge regression model and lasso model to the `Hitters` data. We wish to predict a baseball player's `Salary` based on the statistics associated with performance in the previous year. 

## Import modules, packages and libraries

First, we import some useful modules, packages and libraries. These are needed for carrying out the computations and for plotting the results.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# sci-kit learn specifics
# We will use the sklearn package to obtain ridge regression and lasso models.
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error

## Load the `Hitters` data set

We import the `Hitters` data set as a pandas dataframe.

In [3]:
hitters = pd.read_csv('data/Hitters.csv', index_col = 0)

# Display information about the data set
hitters.info()

# Check for NaN values
# hitters.isna().sum()

# Return summary statistics for each column
# hitters.describe()

# Return first ten rows of the data set
# hitters.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 322 entries, -Andy Allanson to -Willie Wilson
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      322 non-null    int64  
 1   Hits       322 non-null    int64  
 2   HmRun      322 non-null    int64  
 3   Runs       322 non-null    int64  
 4   RBI        322 non-null    int64  
 5   Walks      322 non-null    int64  
 6   Years      322 non-null    int64  
 7   CAtBat     322 non-null    int64  
 8   CHits      322 non-null    int64  
 9   CHmRun     322 non-null    int64  
 10  CRuns      322 non-null    int64  
 11  CRBI       322 non-null    int64  
 12  CWalks     322 non-null    int64  
 13  League     322 non-null    object 
 14  Division   322 non-null    object 
 15  PutOuts    322 non-null    int64  
 16  Assists    322 non-null    int64  
 17  Errors     322 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  322 non-null    obje

From the call on `hitters.info()`, we obtain that the variable `Salary` has only 263 counts which are non-null; this hints to the fact that there might be missing data. Furthermore, we observe that `League`, `Division`, and `NewLeague` are `objects`.

With the code `hitters.isna().sum()`, we confirm that `Salary` indeed has `NaN` values. 
This is also confirmed when we returned the first 20 rows of the dataframe with `hitters.head(20)`; the first row has a `NaN` value for `Salary`. Additionally, we observe that `League` and `NewLeague` have only the values `A` and `N`, and division has the values `S` and `W`.

We need to delete the columns which have missing data for `Salary` and convert the categorical variables.

## Preprocess the `Hitters` data set

In this part, we preprocess the `Hitters` data so that it is ready for the data fitting. 

In [ ]:
# Remove any row containing at least one NaN value
hitters_clean = hitters.dropna(axis = 0, how = 'any')

# Display information about the data set
hitters_clean.info()

# First 10 rows of data set
# hitters_clean.head(10)

We obtain the 'cleaned' data with 263 row and 20 columns. 
This is in agreement with the result in p.244 of the book and also with our earlier observation when calling `hitters.info()`. 

We continue with transforming the *categorical* variables `League`, `Division`, and `NewLeague` to *indicator* variables.

In [ ]:
#create dummies variable
dummies = pd.get_dummies(hitters_clean[['League', 'Division', 'NewLeague']], dtype=int)

# Display information about the data set
# dummies.info()

# Return summary statistics for each column
# dummies.describe()

# First 10 rows of data set
dummies.head()

In [ ]:
# Make a copy of hitters_clean data set
hitters_ind = hitters_clean.copy()

# Replace the columns with their 0/1 values
# League = 'N' is assigned the value 1
# Division = 'W' is assigned the value 1
# NewLeague = 'N' is assigned the value 1
hitters_ind['League'] = dummies['League_N']
hitters_ind['Division'] = dummies['Division_W']
hitters_ind['NewLeague'] = dummies['NewLeague_N']

# Note that hitters_clean is not changed, but hitters_ind is.
hitters_ind.head()

In [ ]:
# Note that hitters_clean is not changed, but hitters_ind is.
hitters_clean.head()

We set up the variables needed for the Ridge Regression and Lasso fit.

In [ ]:
# We set up the variables for the Ridge Regression fit.

# The X variable containing the predictors
X = hitters_ind.drop('Salary', axis=1)

# The y variable containing the response
y = hitters_ind['Salary']

# Standardize the X variable
xscaler = StandardScaler()
X_scaled = xscaler.fit_transform(X)

# Check if X_scaled has mean zero for each column
print('\nMean of the standardized X:')
print(X_scaled.mean(axis = 0))
print('\nVariance of the standardized X:')
print(X_scaled.var(axis = 0))

## Ridge Regression Model

We perform ridge regression in order to predict the baseball player's `Salary` based on the statistics. 

In [ ]:
# Range of values for lambda, the tuning parameter
lambdas = 10**np.linspace(8, -2, 100)

With the chosen range for `lambdas`, we cover the full range of scenarios from the null model containing only the intercept ($ \lambda = 10^{8} $), to the least squares fit ($ \lambda = 10^{-2} $).

For each particular value in `lambdas`, we obtain a vector of ridge regression coefficients + an intercept. 

In [ ]:
ridge = Ridge(fit_intercept=True)

# Create a pandas dataframe to store the coefficients
coefs_ridge = pd.DataFrame(columns=np.append('Intercept', X.columns))

# Loop through lambdas
for i, l in enumerate(lambdas):
    # Update the lambda value (note that the argument is called alpha)
    ridge.set_params(alpha=l)
    ridge.fit(X_scaled, y)
    coefs_ridge.loc[i, :] = np.append([ridge.intercept_], ridge.coef_)

coefs_ridge

In [ ]:
# Plot the coefficients
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(lambdas, coefs_ridge)
ax.set_xscale('log')
ax.set_xlabel('lambda')
ax.set_ylabel('estimated coefficients')
plt.show()

We now look more closely at two specific lambdas, at index $40$ and $60$. We can see from the code below what the $\lambda$ values, coefficients, and magnitude of the penalty term was for these two values.

In [ ]:
# First, we check the coefficients at index 40
idx = 40
l = lambdas[idx]
coefs_idx = coefs_ridge.iloc[idx,:].drop('Intercept')
norm = (coefs_idx**2).sum()**0.5

print('At index {:d}, we find a lambda value of {:.4e}, a penalty norm of {:.4f}, and the following coefficients:'.format(idx, l, norm))
print(coefs_idx)

# Now, we check the coefficients at index 60
idx = 60
l = lambdas[idx]
coefs_idx = coefs_ridge.iloc[idx,:].drop('Intercept')
norm = (coefs_idx**2).sum()**0.5

print('\nAt index {:d}, we find a lambda value of {:.4e}, a penalty norm of {:.4f}, and the following coefficients:'.format(idx, l, norm))
print(coefs_idx)

## Split data into training and test data

We are going to split the data; half of it will be training data while the remaining portion is going to be test data.


In [ ]:
# Obtain training and test data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=1)

Now, we fit a ridge regression model with three different $\lambda$ values ($0$, $4$ and $10^{10}$) using the training data and evaluate its MSE on the test set. The model where $\lambda=0$ corresponds to the least squares fit, whereas $\lambda = 10^{10}$ corresponds to a model with only the intercept.

In [ ]:
lambdas_sel = [0, 4, 10**10]
mse_lambdas_sel = []

# Create a pandas dataframe to store the coefficients
coefs_ridge_sel = pd.DataFrame(columns=np.append('Intercept', X.columns))

# Loop through lambdas
for i, l in enumerate(lambdas_sel):
    ridge.set_params(alpha=l)

    # Use training data to fit the model
    ridge.fit(X_train, y_train)
    coefs_ridge_sel.loc[i, :] = np.append(ridge.intercept_, ridge.coef_)

    # Compute the error based on test data
    mse_lambdas_sel.append(mean_squared_error(y_test, ridge.predict(X_test)))

# Print the MSE for the different values of lambda
print('\n For lambda equal to {}, we obtain a mean squared error of {:.4f}'.format(lambdas_sel[0], mse_lambdas_sel[0]))
print('\n For lambda equal to {}, we obtain a mean squared error of {:.4f}'.format(lambdas_sel[1], mse_lambdas_sel[1]))
print('\n For lambda equal to {:.0e}, we obtain a mean squared error of {:.4f}'.format(lambdas_sel[2], mse_lambdas_sel[2]))

So we see that fitting a ridge regression model with $\lambda = 4$ leads to a much lower test MSE than fitting a model with just an intercept. The MSE score is also smaller than for the least squares fitting without regularization.

## Use Cross-validation to choose the tuning parameter $ \lambda $. 

Until now, we have chosen a specific value for $ \lambda $. We can use 10-fold cross-validation to obtain an optimal value for $ \lambda $. 

In [ ]:
# Optimize alpha with cross-validation
param_grid = {'alpha':lambdas}
ridgeCV = GridSearchCV(ridge, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
ridgeCV.fit(X_train, y_train)

lambda_cv = ridgeCV.best_params_['alpha']
validation_mse = -ridgeCV.best_score_

# Print the cross-validation MSE for the optimized value of lambda
print('\n For lambda equal to {:.4f}, we obtain a mean squared cross-validation error of {:.4f}'.format(lambda_cv, validation_mse))

Using the above code, we obtain the value for $ \lambda $ with the smallest cross-validation error. What is now the test MSE associated to this value of $ \lambda $?

In [ ]:
# Create a pandas dataframe to store the coefficients
coefs_ridge_cv = pd.DataFrame(columns=np.append('Intercept', X.columns))

# Use training data to fit the model
ridge.set_params(alpha=lambda_cv)
ridge.fit(X_train, y_train)
coefs_ridge_cv.loc[0, :] = np.append(ridge.intercept_, ridge.coef_)
ridge_test_mse = mean_squared_error(y_test, ridge.predict(X_test))

# Print the test MSE for the optimized value of lambda
print('\n For lambda equal to {:.4f}, we obtain a mean squared test error of {:.4f}'.format(lambda_cv, ridge_test_mse))

coefs_ridge_cv

Notice that this MSE value is smaller than the MSE value for $\lambda = 4$.
As expected, none of the coefficients are exactly zero - ridge regression does not perform variable selection!

## The Lasso

We have seen that ridge regression with a wise choice of $ \lambda $ can outperform least squares as well as the null model (having only the Intercept) on the `Hitters` data set. 
We now ask whether the lasso can yield either a more accurate or a more interpretable model than ridge regression. 

In [ ]:
lasso = Lasso(fit_intercept=True, max_iter=10000)

# Create a pandas dataframe to store the coefficients
coefs_lasso = pd.DataFrame(columns=np.append('Intercept', X.columns))

# Loop through lambdas
for i, l in enumerate(lambdas):
    lasso.set_params(alpha=l)

    # Use training data to fit the model
    lasso.fit(X_train, y_train)
    coefs_lasso.loc[i, :] = np.append(lasso.intercept_, lasso.coef_)

coefs_lasso

In [ ]:
# Plot the coefficients
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(lambdas, coefs_lasso)
ax.set_xscale('log')
ax.set_xlabel('lambda')
ax.set_ylabel('estimated coefficients')
plt.show()

We can see from the coefficient plot that depending on the choice of tuning parameter, some of the coefficients will be exactly equal to zero. We now perform cross-validation and compute the associated test error.

In [ ]:
# Optimize alpha with cross-validation
param_grid = {'alpha':lambdas}
lassoCV = GridSearchCV(lasso, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
lassoCV.fit(X_train, y_train)

lambda_cv = lassoCV.best_params_['alpha']
validation_mse = -lassoCV.best_score_

# Print the cross-validation MSE for the optimized value of lambda
print('\n For lambda equal to {:.4f}, we obtain a mean squared cross-validation error of {:.4f}'.format(lambda_cv, validation_mse))

# Create a pandas dataframe to store the coefficients
coefs_lasso_cv = pd.DataFrame(columns=np.append('Intercept', X.columns))

# Use training data to fit the model
lasso.set_params(alpha=lambda_cv)
lasso.fit(X_train, y_train)
coefs_lasso_cv.loc[0, :] = np.append(lasso.intercept_, lasso.coef_)
lasso_test_mse = mean_squared_error(y_test, lasso.predict(X_test))

# Print the test MSE for the optimized value of lambda
print('\n For lambda equal to {:.4f}, we obtain a mean squared test error of {:.4f}'.format(lambda_cv, lasso_test_mse))

coefs_lasso_cv

This is lower than the test set MSE of the null model and of least squares, and similar to the test MSE of ridge regression with $\lambda = 4$.
However, the lasso has the advantage over ridge regression in that the resulting coefficient estimates are sparse. Here we see that $2$ of the $19$ coefficient estimates have been set to $0$, so the lasso model with $\lambda=1.0476$ contains only $17$ variables.